In [2]:
# fazendo importações
import pandas as pd
import warnings

In [3]:
# ignorando warnings
warnings.filterwarnings("ignore")

In [4]:
# definindo colunas e tamanho
colspecs = [(2, 10), (10, 22), (56, 69), (82, 95)]
names = ['date', 'cod', 'opening-price', 'closing-price']

In [5]:
# carregando arquivos
df22 = pd.read_fwf('../../bkp-CSVB3/COTAHIST_A2023.TXT', colspecs=colspecs, names=names, header=None)
df23 = pd.read_fwf('../../bkp-CSVB3/COTAHIST_A2022.TXT', colspecs=colspecs, names=names, header=None)
df24 = pd.read_fwf('../../bkp-CSVB3/COTAHIST_A2024.TXT', colspecs=colspecs, names=names, header=None)

In [6]:
# filtrando ações da ambev
ABEV3_df22 = df22[df22['cod'].str.endswith('ABEV3')]
ABEV3_df23 = df23[df23['cod'].str.endswith('ABEV3')]
ABEV3_df24 = df24[df24['cod'].str.endswith('ABEV3')]

In [7]:
# unificando dataframes
df = pd.concat([ABEV3_df22, ABEV3_df23])

In [8]:
# lendo valores
df.head(20)
print(df)

             date      cod  opening-price  closing-price
21       20230102  02ABEV3         1440.0         1402.0
1655     20230103  02ABEV3         1416.0         1411.0
3182     20230104  02ABEV3         1428.0         1408.0
4849     20230105  02ABEV3         1427.0         1414.0
6534     20230106  02ABEV3         1430.0         1416.0
...           ...      ...            ...            ...
1221393  20221202  02ABEV3         1615.0         1595.0
1228746  20221027  02ABEV3         1525.0         1521.0
1229043  20221122  02ABEV3         1570.0         1550.0
1229143  20221111  02ABEV3         1576.0         1566.0
1231359  20221109  02ABEV3         1661.0         1642.0

[498 rows x 4 columns]


In [9]:
# formatando as datas
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# separando e extraindo ano, mês e dia
df['Ano'] = df['date'].dt.year
df['Mes'] = df['date'].dt.month
df['Dia'] = df['date'].dt.day

print(df)

              date      cod  opening-price  closing-price   Ano  Mes  Dia
21      2023-01-02  02ABEV3         1440.0         1402.0  2023    1    2
1655    2023-01-03  02ABEV3         1416.0         1411.0  2023    1    3
3182    2023-01-04  02ABEV3         1428.0         1408.0  2023    1    4
4849    2023-01-05  02ABEV3         1427.0         1414.0  2023    1    5
6534    2023-01-06  02ABEV3         1430.0         1416.0  2023    1    6
...            ...      ...            ...            ...   ...  ...  ...
1221393 2022-12-02  02ABEV3         1615.0         1595.0  2022   12    2
1228746 2022-10-27  02ABEV3         1525.0         1521.0  2022   10   27
1229043 2022-11-22  02ABEV3         1570.0         1550.0  2022   11   22
1229143 2022-11-11  02ABEV3         1576.0         1566.0  2022   11   11
1231359 2022-11-09  02ABEV3         1661.0         1642.0  2022   11    9

[498 rows x 7 columns]
